## 3. STRUKTUR DATA dan KINERJANYA

Implementasi struktur data sertaan Python terdiri atas beberapa macam, fixed structured, array, doubly linked list, hashmap. Spesifikasi interface/class antara lain:  

1. tuple, daftar beragam objek dengan urutan indeks, anggotanya tidak dapat diedit (immutable) 
2. list, daftar beragam objek dengan urutan indeks, anggota dapat diubah, ditambah, dikurangi
3. dict, kumpulan pasangan kunci&nilai, kunci tidak dapat diedit, nilai dapat diedit, anggota dapat dibuang, ditambah
4. set, kumpulan objek tanpa duplikat, tanpa urutan 
5. str, deretan karakter, dengan urutan indeks, tidak dapat diedit

dan beberapa tambahan dari modul pustaka sertaan: 

6. array 
7. double end queue (deque)
8. namedtuple

#### TUPLE

Struktur tuple dapat dibayangkan mirip dengan struktur records pada Pascal atau structs pada C, merupakan sekumpulan kecil data dari berbagai tipe yang dioperasikan sebagai suatu grup. Tuple terdiri atas deretan objek pointer (64-bit pada sistem standar masa kini) yang menunjuk ke berbagai objek di luar tuple itu sendiri, seperti terlihat dari alamat tuple dan alamat elemen-elemennya pada kode berikut.  
Pada sistem 64-bit saya, ternyata ukuran bytes tuple selalu bertambah dengan kelipatan 8 bytes jika memiliki lebih banyak elemen, walaupun dari sembarang tipe, yang ukurannya jauh lebih besar dari 8 bytes.

In [1]:
t1=(65537,)
t2=(65537,'hello')
t3=(65537,'hello',22/7)
t4=(65537,'hello',22/7, True)
t5=(65537,'hello',22/7, True, tuple.count)
print("alamat tuple dibandingkan dengan alamat aggotanya")
for o in t5, *t5:
    print(hex(id(o)),o)
print("\nukuran alokasi memori tuple dibanding ukuran anggotanya")
for o in t1,t2,t3,t4,t5, *t5:
    print(o.__sizeof__(),'bytes', o)

alamat tuple dibandingkan dengan alamat aggotanya
0x7ff0e8f0a340 (65537, 'hello', 3.142857142857143, True, <method 'count' of 'tuple' objects>)
0x7ff0e91d7130 65537
0x7ff0e962ebb0 hello
0x7ff0e8f7c210 3.142857142857143
0x5635baede020 True
0x7ff0ebe3f740 <method 'count' of 'tuple' objects>

ukuran alokasi memori tuple dibanding ukuran anggotanya
32 bytes (65537,)
40 bytes (65537, 'hello')
48 bytes (65537, 'hello', 3.142857142857143)
56 bytes (65537, 'hello', 3.142857142857143, True)
64 bytes (65537, 'hello', 3.142857142857143, True, <method 'count' of 'tuple' objects>)
28 bytes 65537
54 bytes hello
24 bytes 3.142857142857143
28 bytes True
56 bytes <method 'count' of 'tuple' objects>


Ternyata bahwa alamat tuple terpisah jauh dengan alamat objek anggotanya. Besar alokasi memori yang digunakan tuple, tidak cocok dengan besar alokasi memori yang digunakan anggotanya. Alokasi memori yang digunakan tuple terdiri atas overhead 24 bytes ditambah 8 bytes untuk setiap anggotanya.

Tuple adalah immutable, artinya strukturnya fixed, tidak dapat diinsert, didelete, diedit, sehingga proses akses *read only* ke item anggotanya sangat ringan, efisien. Kompleksitas waktunya konstan, O(1). Kinerja tinggi merupakan alasan untuk sebanyak mungkin menggunakan tuple.

In [2]:
t1 = ((2021, 'lulus sekolah'),)
t1 = t1 + ((2022, 'mulai kerja'),)
t1

((2021, 'lulus sekolah'), (2022, 'mulai kerja'))

Pada contoh di atas tuple t1 bukan di edit tetapi dibuatkan objek yang baru yaitu gabungan objek t1 yang sebelumnya dengan suatu ekspresi baru. Kita dapat memastikan dengan membandingkan identitas objeknya.

In [3]:
t1 = ((2021, 'lulus sekolah'),)
old_id = id(t1)
t1 = t1 + ((2022, 'mulai kerja'),)
new_id = id(t1)
old_id, new_id

(140672677661904, 140672677927296)

Kita perlu ingat bahwa operasi binding dalam Python hanya melibatkan *pointer* sedangkan objek aslinya sendiri tidak berpindah, tidak dicopy. Pada contoh berikut objek elemen pertama tetap digunakan pada tuple yang baru, terlihat dari identitas objeknya yang sama. 

In [4]:
t1 = ((2021, 'lulus sekolah'),)
obj0 = id(t1[0])
t1 = t1 + ((2022, 'mulai kerja'),)
obj1 = id(t1[0])
obj0, obj1

(140672677004096, 140672677004096)

In [5]:
t2 = t1[:] #tuple tidak mendukung cloning, selalu aliasing
t1 is t2

True

In [6]:
Lst1 = list([1,2,3,4])
Lst2 = Lst1[:]  # list cloning
Lst1 == Lst2, Lst1 is Lst2

(True, False)

Jika diperhatikan **tuple tidak mendukung cloning**, karena sifatnya yang *immutable* maka tidak ada alasan untuk mendukung cloning, memboroskan memori, membuat objek baru dengan isi yang sama. Jadi walaupun kita menggunakan sintaks yang biasa untuk membuat cloning pada list, efeknya pada tuple adalah aliasing. (cloning adalah membuat objek berbeda berisikan elemen yang sama, sedangkan aliasing adalah memberi nama yang berbeda untuk objek yang sama)

In [7]:
'''
tuple, kumpulan objek yang dapat diiterasi
generator expressions, mirip list comprehension tapi menghasilkan objek generator
unpacking operator asterisk * memiliki efek mengiterasi generator
'''
for klas in tuple, list:
    metodas = (m for m in dir(klas) if '_' not in m)   # membentuk suatu generator
    print(metodas)
    print(*metodas,'\n')   # asterisk * unpacking generator, mengeluarkan elemen elemennya.


<generator object <genexpr> at 0x7ff0e8fdcac0>
count index 

<generator object <genexpr> at 0x7ff0e8fdca50>
append clear copy count extend index insert pop remove reverse sort 



Iterable ialah suatu objek yang mampu mengembalikan elemen anggotanya satu per satu. Contoh iterable antara lain semua tipe sekuens (seperti list, tuple, str), dan beberapa tipe non-sekuens seperti dict, objek file, dan juga objek dari klas yang memiliki metoda \_\_iter\_\_() atau \_\_getitem\_\_(), yang mengimplementasikan semantik sekuens.   
   
Iterable dapat digunakan bersama for loop atau bersama fungsi lain yang memerlukan sekuens seperti map(), zip().   
  
Generator ialah suatu fungsi yang mengeluarkan elemen di dalamnya satu per satu, dan cenderung lebih hemat memori daripada list yang dibentang isi elemennya. Generator dapat digunakan bersama for loop, atau fungsi next() yang hanya mengeluarkan satu elemen setiap kali dipanggil. (ekspresi bintang, asterisk * juga dapat digunakan dalam kasus tertentu)

#### LIST

List adalah suatu array yang ukurannya dapat berubah ubah. Implementasinya menempati satu lokasi memori yang utuh tidak terpecah, berisi pointer pointer objek yang diletakkan berdampingan, dan pada kepala list tercantum pointer ke lokasi ini, serta jumlah elemen atau panjang list.   
Pada akhir dari lokasi array sudah dicadangkan tempat untuk pointer baru jika anggota array bertambah, dan jika cadangan ini habis maka akan dialokasikan tempat (bisa jadi dengan alamat baru) dengan cadangan yang lebih besar lagi, karena setiap alokasi tempat baru akan memakan ongkos waktu untuk menyalin semua pointer yang ada dari lokasi lama ke dalamnya.    
Proses mencari anggota dalam list (pointer ke objek) dapat dilakukan langsung tanpa tergantung dari jumlah anggota list, karena alamatnya dapat diperoleh dengan perkalian indeks dan ukuran pointer ditambah offset awal dari lokasi, sama seperti mengakses RAM. Notasi kompleksitasnya adalah konstan O(1).
Penambahan anggota di akhir list, dapat dianggap kecil juga karena ongkos tambahan penyalinan di-amortisasi, ditanggung bersama anggota-anggota yang ditambahkan langsung selama cadangan belum habis terpakai, lalu ukuran cadangan yang makin membesar setiap kali dibuat baru.   


In [8]:
list1 = list()
list2 = [28]
list3 = [28,42]
list5 = [28,42,255,'list array of pointers']

for obj in list5, *list5:
    print(hex(id(obj)))
for obj in list1,list2,list3:
    print(obj.__sizeof__())


0x7ff0e45f74c0
0x7ff0ebde0450
0x7ff0ebde0610
0x7ff0ebde20b0
0x7ff0e8fea600
40
48
56


#### ARRAY

In [36]:
from array import array
arr1 = array('b',[1])
arr2 = array('b',[1,2])
arr3 = array('b',[1,2,3])
arr4 = array('b',[1,2,3,4])
arr5 = array('b',[1,2,3,4,5])
for o in arr5, arr5[0], arr5[1],arr5[2]:
    print(hex(id(obj)))
print(f'{arr5.itemsize = } byte(s)')
for o in arr1, arr2, arr3, arr4, arr5:
    print(o.__sizeof__(), 'bytes, untuk', o)
print('dalam proses append ternyata ada tambahan reservasi memori')
for i in range(6,12):
    arr5.append(i)
    print(arr5.__sizeof__(), 'bytes, untuk',arr5)

0x7ff0e45f7640
0x7ff0e45f7640
0x7ff0e45f7640
0x7ff0e45f7640
arr5.itemsize = 1 byte(s)
65 bytes, untuk array('b', [1])
66 bytes, untuk array('b', [1, 2])
67 bytes, untuk array('b', [1, 2, 3])
68 bytes, untuk array('b', [1, 2, 3, 4])
69 bytes, untuk array('b', [1, 2, 3, 4, 5])
dalam proses append ternyata ada tambahan reservasi memori
73 bytes, untuk array('b', [1, 2, 3, 4, 5, 6])
73 bytes, untuk array('b', [1, 2, 3, 4, 5, 6, 7])
73 bytes, untuk array('b', [1, 2, 3, 4, 5, 6, 7, 8])
73 bytes, untuk array('b', [1, 2, 3, 4, 5, 6, 7, 8, 9])
81 bytes, untuk array('b', [1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
81 bytes, untuk array('b', [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])


Implementasi array.array ini berupa deretan objek homogen statik, langsung, bukan pointer, terlihat dari alamat dan penambahan ukuran array seperti pada kode di atas. Alamat anggota array adalah sama dengan alamat kepala array ('serumah'). Deklarasi tipe  atau ukuran anggota, dalam contoh ini 'b' adalah untuk signed integer 1 byte minimum. Bahkan kita dapat melihat penambahan cadangan di akhir array, tambah 4, tambah 8, makin membesar. Kompleksitas waktu sama dengan list, kompleksitas ruang sama, tetapi pemakaian absolut memori lebih hemat, dapat diatur secara statik.


#### DEQUE

Menurut seorang core developer Python, Raymond Hettinger, proses insert elemen di posisi awal pada suatu list menggunakan waktu yang tidak sedikit, O(n), sehingga jika suatu program besar banyak melakukan hal itu maka kinerja keseluruhan akan menurun. Hal ini disebabkan karena semua elemen harus digeser satu per satu untuk menyediakan tempat kosong di posisi awal. Itulah sebabnya dia menambahkan suatu struktur data doubly-linked list, double ended queue atau disingkat deque dengan metoda andalannya *appendleft, extendleft, popleft*, dalam modul pustaka collections.   
Operasi append, pop pada posisi awal/akhir deque memiliki kompleksitas waktu konstan O(1).
Tetapi kinerja deque melambat untuk akses elemen di posisi pertengahan, karena harus menelusuri linked list.    
Contoh berikut menampilkan kontras perbandingan kinerja deque terhadap list pada proses insert di posisi awal.

In [10]:
from time import time
from collections import deque

deck = deque()
start = time()
for n in range(1_000_000):
    deck.appendleft(n)
stop = time()
print(f'deque appendleft(n) {1+deck[0]:,} elemen {stop-start:.2f} detik')

Lst = list()
nmax = 250_000
start = time()
print(f'mulai {start} ... please wait...')
for n in range(nmax):
    Lst.insert(0,n)
stop = time()
print(f'selesai {stop}')
print(f'list insert(0,n) {1+Lst[0]:,} elemen {stop-start:.2f} detik')


deque appendleft(n) 1,000,000 elemen 0.18 detik
mulai 1661066503.054439 ... please wait...
selesai 1661066524.0881388
list insert(0,n) 250,000 elemen 21.03 detik


#### DICT dan SET

Struktur dari dict/set adalah hashmap atau sebutan lainnya hashtable. Dict adalah kumpulan objek dengan tatanan berupa pasangan kunci dan nilai, Set adalah kumpulan objek tanpa duplikasi.  
Kunci pada dictionary haruslah objek yang hashable karena strukturnya tidak memiliki indeks tetapi menggunakan fungsi hash pada key/kunci sebagai 'indeks' elemennya. Objek yang hashable antara lain integer, float, string, tuple yang beranggotakan hashable object. List termasuk objek yang unhashable, tidak dapat digunakan sebagai kunci.  
Dalam hashmap/dict kompleksitas pencarian elemen adalah O(1) dan insertion/deletion juga O(1), artinya konstan tidak tergantung pada banyaknya elemen anggota yang terdaftar.  
Jika terjadi *collision* atau fungsi hash atas dua atau beberapa key menghasilkan 'indeks hash' yang sama maka perlu suatu penanganan lagi, misalnya perbaikan fungsi hash, linear probing, quadratic probing, double hash layering, atau menggunakan chaining link list, dan ini akan mempengaruhi performa kompleksitas waktunya.

In [33]:
nomor_telp = dict()
nomor_telp[('sam','edw')] = '0888999xxxx'
nomor_telp[('budi','di amrik')] = '(818) 785 xxxx'
print(nomor_telp[('sam','edw')])
print(nomor_telp)

0888999xxxx
{('sam', 'edw'): '0888999xxxx', ('budi', 'di amrik'): '(818) 785 xxxx'}


Salah satu penggunaan dict ialah untuk menghitung jumlah pemunculan huruf huruf. Berikut contoh kode tentang anagram, permainan kata, yaitu huruf huruf dalam suatu kata kata diacak menjadi kata kata yang berbeda, misalnya beras|sebar|besar, "eleven plus two" | "twelve plus one", "drum piano asbak"|"opa bandar musik". Kode kita hanya akan menghitung huruf dan menyimpulkan apakah dua ungkapan kata merupakan anagram atau tidak.

In [12]:
def anagram(word,words):
    def cc(word):
        d={}
        for c in word.lower().replace(' ',''):
            if c not in d:
                d[c] = 0
            d[c] = d[c] + 1
        return d
    ref = cc(word)
    return [wrd for wrd in words if cc(wrd) == ref]
    
print(anagram("setec astronomy", ("too many secrets", "my Socrates note")))
print(anagram('beras',('besar','sebar', 'sabar','resah')))


['too many secrets', 'my Socrates note']
['besar', 'sebar']


Baris ke-7 pada contoh di atas menghitung kemunculan setiap huruf dalam dictionary dengan kunci berupa hurufnya dan nilainya diakumulasi. Baris ke-5,6,7 dapat diganti menjadi satu baris menggunakan metoda dict.get() sbb:

In [13]:
def anagram(word,words):
    def cc(word):
        d={}
        for c in word.lower().replace(' ',''):
            d[c] = d.get(c,0) + 1
        return d
    ref = cc(word)
    return [wrd for wrd in words if cc(wrd) == ref]
    
print(anagram("setec astronomy", ("too many secrets", "my Socrates note")))
print(anagram('beras',('besar','sebar', 'sabar','resah')))


['too many secrets', 'my Socrates note']
['besar', 'sebar']


Proses counting dengan dictionary sepertinya banyak dipakai sehingga 'seseorang' (bisa jadi Guido sendiri) membuat class khusus bernama Counter yang juga merupakan implementasi dari bentuk matematis **multiset**, dan disematkan dalam pustaka collections. Akhirnya, kode kita dapat dipadatkan menjadi:

In [32]:
from collections import Counter
print(Counter)
def anagram(word,words):
    cc = lambda w: Counter(w.lower().replace(' ',''))
    ref = cc(word)
    return word, *(wrd for wrd in  words if cc(wrd) == ref)
    
print(anagram("setec astronomy", ("too many secrets", "my Socrates note")))
print(anagram('beras',('besar','sebar', 'beres','resap')))


<class 'collections.Counter'>
('setec astronomy', 'too many secrets', 'my Socrates note')
('beras', 'besar', 'sebar')


Kita dapat melihat gaya penulisan fungsional ala Python seperti contoh berikut yang melakukan pengecekan apakah dua ungkapan merupakan anagram atau bukan.

In [15]:
from collections import Counter
cc = lambda w: Counter(w.lower().replace(' ',''))
is_anagram = lambda w1,w2: cc(w1) == cc(w2)

print(is_anagram('eleven plus two','twelve plus one'))

True


Pustaka Counter ditulis dengan Python sehingga kode sumbernya dapat dibaca sebagai berikut:     
`from collections import Counter`    
`import inspect`    
`print(inspect.getsource(Counter))`    
Pustaka yang ditulis dengan bahasa C, tidak bisa lagi dibaca kode sumbernya.

**----o0o----**